In [76]:
#using jupyter notebook in EMR cluster to test the results
#importing all necessary libraries
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.functions import monotonically_increasing_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#configurations
#cant connect for some reason
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=''
os.environ['AWS_SECRET_ACCESS_KEY']=''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
#reading song data file
input_data = "s3a://udacity-dend/"
song_data = 'song_data/*/*/*/*.json'

df = spark.read.json(input_data+song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
#extract columns to create songs table
df.createOrReplaceTempView('song_data_table')
df.select("song_id", "title", "artist_id", "year", "duration").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+----------+
|           song_id|               title|         artist_id|year|  duration|
+------------------+--------------------+------------------+----+----------+
|SOVIYJY12AF72A4B00|The Dead Next Doo...|AR4T2IF1187B9ADBB7|1983| 233.22077|
|SOVYXYL12AF72A3373|Rebel Yell (1999 ...|AR4T2IF1187B9ADBB7|1983| 287.92118|
|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...|ARQ846I1187B9A7083|   0| 196.04853|
|SOLQYSZ12AB0181F97|    Mony Mony (Live)|AR4T2IF1187B9ADBB7|1987| 247.53587|
|SOVPFJK12A6701CB16|Barcelona - (Frie...|AR3TZ691187FB3DBB1|2000| 273.44934|
|SOETDBF12A81C20BC0|       Body Snatcher|AR4T2IF1187B9ADBB7|2005| 237.06077|
|SOIEXLS12A6D4F792F|           Shangrila|AR4T2IF1187B9ADBB7|1993| 443.14077|
|SOQEBML12A8C136AA4|Werther (1997 Dig...|AR5R7791187FB3A8C3|   0|  55.40526|
|SOTCIHX12A8C13DDD2|Finally_ as that ...|ARYOIZG1187FB41E30|2006| 483.34322|
|SODUMDU12AC468A22B|We're Skrewed (Ot...|ARHOSMU1242078130D|   0|  249.5473|

In [46]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [44]:
#extracting columns to create artists table
df.createOrReplaceTempView("artist_table")

df.select("artist_id",'artist_name','artist_location','artist_latitude','artist_longitude').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|AR4T2IF1187B9ADBB7|          Billy Idol|<a href="http://b...|       63.96027|        10.22442|
|AR4T2IF1187B9ADBB7|          Billy Idol|<a href="http://b...|       63.96027|        10.22442|
|ARQ846I1187B9A7083|Yvonne S. Moriart...|                    |           null|            null|
|AR4T2IF1187B9ADBB7|          Billy Idol|<a href="http://b...|       63.96027|        10.22442|
|AR3TZ691187FB3DBB1|Russell Watson / ...|                    |           null|            null|
|AR4T2IF1187B9ADBB7|          Billy Idol|<a href="http://b...|       63.96027|        10.22442|
|AR4T2IF1187B9ADBB7|          Billy Idol|<a href="http://b...|       63.96027|        10.22442|
|AR5R7791187FB3A8C3|Alfredo Kraus/Tat...

In [54]:
#reading log_data
input_data = 's3a://udacity-dend/'
log_data = 'log_data/*/*/*.json'

df1 = spark.read.json(input_data+log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
#filter by actions for song plays
df1 = df1.filter(df1.page == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
#extracting columns for users table
df1.createOrReplaceTempView("users_table")
df1.select('userId','firstName','lastName','gender','level').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    61|   Samuel|Gonzalez|     M| free|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    26|     Ryan|   Smith|     M| free|
|    26|     Ryan|   Smith|     M| free|
|    49|    Chloe|  Cuevas|     F| paid|
+------+---------+--------+------+-----+
only showing top

In [52]:
df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: double, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [11]:
df1.select('ts').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|           ts|
+-------------+
|1542241826796|
|1542242481796|
|1542242741796|
|1542247071796|
|1542252577796|
|1542253449796|
|1542253460796|
|1542260074796|
|1542260277796|
|1542260935796|
|1542261224796|
|1542261356796|
|1542261662796|
|1542261713796|
|1542262057796|
|1542262233796|
|1542262434796|
|1542262456796|
|1542262679796|
|1542262728796|
+-------------+
only showing top 20 rows

In [67]:
from pyspark.sql.types import TimestampType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
#creating timestamp column from original ts column
get_timestamp = udf(lambda x: datetime.fromtimestamp (x // 1000), TimestampType())
df1 = df1.withColumn('datetime', get_timestamp(col('ts')))
df1.select('datetime').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|           datetime|
+-------------------+
|2018-11-15 00:30:26|
|2018-11-15 00:41:21|
|2018-11-15 00:45:41|
|2018-11-15 03:44:09|
|2018-11-15 05:48:55|
|2018-11-15 05:53:44|
|2018-11-15 05:55:56|
|2018-11-15 06:01:02|
|2018-11-15 06:07:37|
|2018-11-15 06:10:33|
|2018-11-15 06:13:54|
|2018-11-15 06:14:16|
|2018-11-15 06:17:59|
|2018-11-15 06:18:48|
|2018-11-15 06:21:33|
|2018-11-15 06:25:58|
|2018-11-15 06:29:38|
|2018-11-15 07:08:36|
|2018-11-15 07:12:09|
|2018-11-15 07:28:47|
+-------------------+
only showing top 20 rows

In [69]:
#extracting columns to create time table
df1.selectExpr("datetime as start_time", 'hour(datetime) as hour',
                             'day(datetime) as day', 'weekofyear(datetime) as week',
                             'month(datetime) as month', 'year(datetime) as year',
                            'dayofweek(datetime) as weekday').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:26|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:41:21|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:45:41|   0| 15|  46|   11|2018|      5|
|2018-11-15 03:44:09|   3| 15|  46|   11|2018|      5|
|2018-11-15 05:48:55|   5| 15|  46|   11|2018|      5|
|2018-11-15 05:53:44|   5| 15|  46|   11|2018|      5|
|2018-11-15 05:55:56|   5| 15|  46|   11|2018|      5|
|2018-11-15 06:01:02|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:07:37|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:10:33|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:13:54|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:14:16|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:17:59|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:18:48|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:21:33|   6| 15|  46|   11|2018|      5|
|2018-11-1

In [57]:
df.printSchema()
df1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = 

In [ ]:
#read in song data to use for songplays table
#extract columns from joined song and log datasets to create songplays table
song_df = df
condition = [song_df.artist_name == df1.artist, song_df.title == df1.song, song_df.duration == df1.length]
songplays_table = df1.join(song_df, condition, 'inner') \
                  .selectExpr ('datetime as start_time', 'userId as user_id','level as level','song_id as song_id',
                               'artist_id as artist_id','sessionId as session_id','artist_location as location',
                               'userAgent as user_agent')
songplays_table = songplays_table.withColumn('songplay_id', monotonically_increasing_id())
songplays_table.show()

In [ ]:
#writing to parquet files to S3 buckets
df.write.parquet('s3a://aws-emr-resources-045504181145-us-west-2/songs_table.parquet', partitionBy = ('year','artist_id'), mode = 'overwrite')